The goal of this ML task is to predict the Berg Balance Score for a patient from the tasks they have completed, with data collected from 2D video recordings collected using the trt_pose model.

Strategy for feature engineering:
For each person, get a "score" from each joint from each task, and given that persons score train the model 
For 18 joints across 14 tasks, we thus need to assign 252 weights.

The exact method of deriving the score will most likely be a measure of the covariance or jerkiness of the joint movement.



trt_pose first created a csv of data for each video. For each patient, the csvs for each task were combined by adding two extra columns, a columns for the frame number, as well as a column denoting the task that frame a part of. We can therefore avoid 3D/Multiindexed dataframes etc. 


Y value (Berg score will be last column)?


In [19]:
import pandas as pd






In [27]:
#code to find center of mass of human given frame points from trt_pose
import numpy as np

def centerOfMass(listofPoints): 
    COM=np.array([0,0])
    print(COM)
#[nose,left_eye,right_eye,left_ear,right_ear,left_shoulder,right_shoulder, 0-6
# left_elbow,right_elbow,left_wrist,right_wrist,left_hip,right_hip,left_knee, 8-14
# right_knee,left_ankle,right_ankle,neck] 15-18
    head = np.array(listofPoints[0:5]+[listofPoints[17]])
    
    leftArm = np.array([listofPoints[5],listofPoints[7],listofPoints[9]])#shoulder, elbow, wrist
    rightArm = np.array([listofPoints[6]]+[listofPoints[8]]+[listofPoints[10]])
    
    leftLeg = np.array([listofPoints[11],listofPoints[13],listofPoints[15]])# hip, knee, ankle
    rightLeg = np.array([listofPoints[12],listofPoints[14],listofPoints[16]])
    
    #the thorax is defined as 25% of the way down from the shoulders to the hips
    thorax = [np.average([listofPoints[5],listofPoints[11]],weights=[0.75,0.25],axis=0),
            np.average([listofPoints[6],listofPoints[12]],weights=[0.75,0.25],axis=0)]
    #the abdomen is defined as 75% of the way down from the shoulders to the hips
    abdomen = [np.average([listofPoints[5],listofPoints[11]],weights=[0.25,0.75],axis=0),
            np.average([listofPoints[6],listofPoints[12]],weights=[0.75,0.25],axis=0)]

    pelvis = np.average([listofPoints[11],listofPoints[12]],axis=0)

    COM= np.add(COM,0.0823*np.average(head,axis=0))#head and neck are 8.23% of human mass
    print(COM)
    COM= np.add(COM,0.0307*np.average(leftArm[:-1],axis=0)) # upper arms are 3.07% human mass
    COM= np.add(COM,0.0307*np.average(rightArm[:-1],axis=0))
    COM= np.add(COM,0.023*np.average(leftArm[1:],axis=0)) # forearms +hands are 2.3% human mass
    COM= np.add(COM,0.023*np.average(rightArm[1:],axis=0))
    print(COM)
    COM= np.add(COM,0.111*np.average(leftLeg[:-1],axis=0)) # upper legs are 11.1% human mass
    COM= np.add(COM,0.111*np.average(rightLeg[:-1],axis=0))
    COM= np.add(COM,0.064*np.average(leftLeg[1:],axis=0)) # shank+foot + are 6% human mass
    COM= np.add(COM,0.064*np.average(rightLeg[1:],axis=0)) 
    print(COM)
    COM= np.add(COM,0.185*np.average(thorax,axis=0))
    COM= np.add(COM,0.126*np.average(abdomen,axis=0))
    COM= np.add(COM,0.148*np.average(pelvis,axis=0))
    return COM




    

In [21]:
import numpy as np

array = np.array([[3,0],[1,3]])
print(array)

[[3 0]
 [1 3]]


In [22]:
print(np.average(array,axis=0))

[2.  1.5]


In [28]:
#example frame
array = [[0.45653706789016724, 0.5874448418617249], [0.4494599997997284, 0.5914446115493774], [0.4498015344142914, 0.5816666483879089], [0.45302242040634155, 0.5983349084854126], [0.45332369208335876, 0.5725973844528198], [0.49626025557518005, 0.6071421504020691], [0.4967309534549713, 0.5596976280212402], [0.5454670190811157, 0.6109985709190369], [0.5435549020767212, 0.5496378540992737], [0.5388516783714294, 0.5826149582862854], [0.5372021198272705, 0.573470413684845], [0.601287305355072, 0.5983958840370178], [0.6024929285049438, 0.5624164342880249], [0.6627643704414368, 0.6260895729064941], [0.6713545918464661, 0.557211697101593], [0.7640379071235657, 0.6225649118423462], [0.7726675271987915, 0.5630196928977966], [0.4961824417114258, 0.583387553691864]]
print(centerOfMass(array))

[0 0]
[0.03783505 0.04821238]
[0.09469233 0.11058144]
[0.32741211 0.31648409]
[0.58081495 0.58490778]
